In [3]:
# libraries 
import geopandas as gpd
import numpy as np 
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
import os
import json
from shapely.geometry import box, Polygon
import logging
import random
import torchvision.transforms.functional as TF
import torch

def generate_geoseries( bounds, crs, dims):

    """
    Generates all dim x dim polygons present in the hillshade TIF.
    """

    # height and width of new squares 
    square_dims = dims # 100x100 meters

    # Calculate the number of segments in each dimension (tif width // desired width in pixels!)
    segments_x = 2500 // square_dims # for depth data its 2500
    segments_y = 2500 // square_dims

    # Create an empty list to store the polygons
    polygons = []

    # Iterate over the segments
    for i in range(segments_y):
        for j in range(segments_x):
            # Calculate the coordinates of the segment
            left = bounds.left + j * square_dims
            bottom = bounds.bottom + i * square_dims
            right = left + square_dims
            top = bottom + square_dims

            # Create a polygon for the segment
            polygon = Polygon([(right, bottom), (left, bottom), (left, top), (right, top), (right, bottom)])

            # Append the polygon to the list
            polygons.append(polygon)

    # Create a GeoSeries from the list of polygons
    return gpd.GeoSeries(polygons, crs=crs)

In [17]:
img = rasterio.open('/home/nadjaflechner/Palsa_data/filtered_tifs/hillshade/749_68_2575_2014.tif')
shape_file = "/home/nadjaflechner/Palsa_data/Ground_truth/gt_polygons.shp"
groundtruth_polygs = gpd.read_file(shape_file)

generated_polygons_all = generate_geoseries(img.bounds, img.crs, 200)


In [18]:
d = {'name': [i for i in range(len(generated_polygons_all))]}
generated_polygons_all_df = gpd.GeoDataFrame(d, geometry = generated_polygons_all, crs=generated_polygons_all.crs)

In [19]:
generated_polygons_all.index

RangeIndex(start=0, stop=144, step=1)

In [22]:

for idx, polygon in zip(generated_polygons_all_df.index, generated_polygons_all_df.geometry):

    # see if there is an overlap with ground truths:
    polyg_df = generated_polygons_all_df.iloc[[idx]]
    intersections = gpd.overlay(groundtruth_polygs, polyg_df, how='intersection')

    # only crop if theres a ground truth overlap
    if not intersections.empty:
        print('overlap')
        break


overlap


In [27]:
intersections.geometry[0].bounds[0]

688149.8104874288

In [32]:
minx_ruta = intersections.geometry[0].bounds[0]
miny_ruta = intersections.geometry[0].bounds[1]

miny = str(miny_ruta)[0:3]
minx = str(minx_ruta)[0:2]

if 0 <= int(str(miny_ruta)[3:5]) < 25:
    km_siffran_y = '00'
elif 25 <= int(str(miny_ruta)[3:5]) < 50:
    km_siffran_y = '25'
elif 50 <= int(str(miny_ruta)[3:5]) < 75:
    km_siffran_y = '50'
elif 75 <= int(str(miny_ruta)[3:5]) < 100:
    km_siffran_y = '75'

if 0 <= int(str(minx_ruta)[3:5]) < 25:
    km_siffran_x = '00'
elif 25 <= int(str(minx_ruta)[3:5]) < 50:
    km_siffran_x = '25'
elif 50 <= int(str(minx_ruta)[3:5]) < 75:
    km_siffran_x = '50'
elif 75 <= int(str(minx_ruta)[3:5]) < 100:
    km_siffran_x = '75'

year = 2018 # WHICH YEAR SHOULD IT BE??

filename = f"{miny}_{minx}_{km_siffran_y}{km_siffran_x}_{year}.tif"

In [34]:
print(minx_ruta)
int(str(minx_ruta)[3:5])

688149.8104874288


14

filename

In [39]:
minx_ruta = 688041
miny_ruta = 7492714

miny = str(miny_ruta)[0:3]
minx = str(minx_ruta)[0:2]

if 0 <= int(str(miny_ruta)[3:5]) < 25:
    km_siffran_y = '00'
elif 25 <= int(str(miny_ruta)[3:5]) < 50:
    km_siffran_y = '25'
elif 50 <= int(str(miny_ruta)[3:5]) < 75:
    km_siffran_y = '50'
elif 75 <= int(str(miny_ruta)[3:5]) < 100:
    km_siffran_y = '75'

if 0 <= int(str(minx_ruta)[2:4]) < 25:
    km_siffran_x = '00'
elif 25 <= int(str(minx_ruta)[2:4]) < 50:
    km_siffran_x = '25'
elif 50 <= int(str(minx_ruta)[2:4]) < 75:
    km_siffran_x = '50'
elif 75 <= int(str(minx_ruta)[2:4]) < 100:
    km_siffran_x = '75'

year = 2018 # WHICH YEAR SHOULD IT BE??

filename = f"{miny}_{minx}_{km_siffran_y}{km_siffran_x}_{year}.tif"

In [36]:
filename

'749_68_2575_2018.tif'